In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['gender_submission.csv', 'test.csv', 'train.csv']


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
def gender(sex):
    if sex=='male':
        return 1
    elif sex=='female':
        return 0
def port(emb):
    if emb=='C':
        return 1
    elif emb=='Q':
        return 2
    elif emb=='S':
        return 3
train['Sex'] = train.apply(lambda x : gender(x['Sex']),axis=1)
test['Sex'] = test.apply(lambda x: gender(x['Sex']),axis=1)
train.drop(['Name','Ticket','Cabin','Embarked'],axis=1,inplace=True)
test.drop(['Name','Ticket','Cabin','Embarked'],axis=1,inplace=True)
#train['Embarked'] = train.apply(lambda x:port(x['Embarked']),axis=1)
#test['Embarked'] = test.apply(lambda x:port(x['Embarked']),axis=1)
train['Age'].fillna(int(np.mean(train['Age'])),inplace=True)
test['Age'].fillna(int(np.mean(test['Age'])),inplace=True)
train.set_index('PassengerId',inplace=True)
test.set_index('PassengerId',inplace=True)

In [3]:
y = train['Survived']
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
x = train[features]
X_train,X_test,y_train,y_test = train_test_split(x,y,random_state=0)

In [4]:
clf = RandomForestClassifier()
grid_values = param_grid = { 
    'n_estimators': [100],
    'max_features': ['log2'],
    'max_depth' : [5],
    'criterion' :['gini'],
    'min_samples_leaf' :[5],
    'min_samples_split' :[12],
}
grid_clf = GridSearchCV(clf, param_grid = grid_values)
grid_clf.fit(X_train,y_train)
grid_clf.score(X_test,y_test)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8161434977578476

In [5]:
final_test = test.copy()
final_survived = grid_clf.predict(final_test)
answer = pd.DataFrame({'PassengerID':test.index,'Survived':final_survived})
answer.to_csv('try_1.csv',index=False)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').